In [3]:
import pandas as pd
import cvxpy as cp
import numpy as np
from numpy import random

# File path will be different on another device and must be replaced with the correct file path.
students = pd.read_csv("Sample Data (for scheduling).csv")
print(students.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               60 non-null     int64 
 1   First Name       60 non-null     object
 2   Last Name        60 non-null     object
 3   Gender           60 non-null     object
 4   Age              60 non-null     int64 
 5   Times Available  60 non-null     object
dtypes: int64(2), object(4)
memory usage: 2.9+ KB
None


In [6]:
students['Age'].value_counts()

Age
9     13
12    12
11    11
13     9
8      8
10     7
Name: count, dtype: int64

In [7]:
# Reference: https://www.geeksforgeeks.org/create-a-new-column-in-pandas-dataframe-based-on-the-existing-columns/
def grade(age):
    if age > 11:
        return "elem"
    else:
        return "ms"

students['Grade'] = students['Age'].map(grade)
display(students.head())

,ID,First Name,Last Name,Gender,Age,Times Available,Grade
0,0,Ximeno,Colombo,M,9,"[1, 2, 3, 4]",ms
1,1,Dorothea,Clayton,F,9,"[1, 3, 4]",ms
2,2,Beibhinn,Nye,F,13,"[1, 2, 3]",elem
3,3,Oskar,Ciobanu,M,12,"[1, 2, 3, 4]",elem
4,4,Matilda,Solomon,F,11,"[2, 3]",ms


In [8]:
students['Grade'].value_counts()

Grade
ms      39
elem    21
Name: count, dtype: int64

In [11]:
firstHour = []
secondHour = []
thirdHour = []
fourthHour = []

for i in students['Times Available']:
    x = []
    for j in i:
        if j.isdigit(): # Reference: https://www.w3schools.com/python/ref_string_isdigit.asp
            x.append(int(j))
    if 1 in x:
        firstHour.append(1)
    else:
        firstHour.append(0)
    if 2 in x:
        secondHour.append(1)
    else:
        secondHour.append(0)
    if 3 in x:
        thirdHour.append(1)
    else:
        thirdHour.append(0)
    if 4 in x:
        fourthHour.append(1)
    else:
        fourthHour.append(0)

# Reference: https://www.geeksforgeeks.org/add-multiple-columns-to-dataframe-in-pandas/
students['First Hour'] = firstHour
students['Second Hour'] = secondHour
students['Third Hour'] = thirdHour
students['Fourth Hour'] = fourthHour

display(students.head())

,ID,First Name,Last Name,Gender,Age,Times Available,Grade,First Hour,Second Hour,Third Hour,Fourth Hour
0,0,Ximeno,Colombo,M,9,"[1, 2, 3, 4]",ms,1,1,1,1
1,1,Dorothea,Clayton,F,9,"[1, 3, 4]",ms,1,0,1,1
2,2,Beibhinn,Nye,F,13,"[1, 2, 3]",elem,1,1,1,0
3,3,Oskar,Ciobanu,M,12,"[1, 2, 3, 4]",elem,1,1,1,1
4,4,Matilda,Solomon,F,11,"[2, 3]",ms,0,1,1,0


In [12]:
# Reference: https://www.statology.org/pandas-sum-column-with-condition/
print("Number of elementary students who can attend first hour:", students.loc[students['Grade'] == "elem", 'First Hour'].sum())
print("Number of middle school students who can attend first hour:", students.loc[students['Grade'] == "ms", 'First Hour'].sum())

print("\nNumber of elementary students who can attend first hour:", students.loc[students['Grade'] == "elem", 'Second Hour'].sum())
print("Number of middle school students who can attend first hour:", students.loc[students['Grade'] == "ms", 'Second Hour'].sum())

print("\nNumber of elementary students who can attend first hour:", students.loc[students['Grade'] == "elem", 'Third Hour'].sum())
print("Number of middle school students who can attend first hour:", students.loc[students['Grade'] == "ms", 'Third Hour'].sum())

print("\nNumber of elementary students who can attend first hour:", students.loc[students['Grade'] == "elem", 'Fourth Hour'].sum())
print("Number of middle school students who can attend first hour:", students.loc[students['Grade'] == "ms", 'Fourth Hour'].sum())

Number of elementary students who can attend first hour: 17
Number of middle school students who can attend first hour: 25

Number of elementary students who can attend first hour: 12
Number of middle school students who can attend first hour: 18

Number of elementary students who can attend first hour: 11
Number of middle school students who can attend first hour: 26

Number of elementary students who can attend first hour: 13
Number of middle school students who can attend first hour: 23


In [17]:
random.seed(3230)
trouble = random.choice([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], size=len(students)) # Reference: https://www.w3schools.com/python/numpy/numpy_random.asp
students['Trouble'] = trouble # Reference: https://www.geeksforgeeks.org/adding-new-column-to-existing-dataframe-in-pandas/

disability = random.choice([0,1], size=len(students))
students['Disbility'] = disability

display(students.head())

,ID,First Name,Last Name,Gender,Age,Times Available,Grade,First Hour,Second Hour,Third Hour,Fourth Hour,Trouble,Disbility
0,0,Ximeno,Colombo,M,9,"[1, 2, 3, 4]",ms,1,1,1,1,10,1
1,1,Dorothea,Clayton,F,9,"[1, 3, 4]",ms,1,0,1,1,10,1
2,2,Beibhinn,Nye,F,13,"[1, 2, 3]",elem,1,1,1,0,7,0
3,3,Oskar,Ciobanu,M,12,"[1, 2, 3, 4]",elem,1,1,1,1,6,1
4,4,Matilda,Solomon,F,11,"[2, 3]",ms,0,1,1,0,7,1


In [18]:
# Reference: https://www.geeksforgeeks.org/split-pandas-dataframe-by-column-value/
elementary = students[students['Grade'] == "elem"]
middle = students[students['Grade'] == "ms"]
# Reference: https://www.geeksforgeeks.org/python-delete-rows-columns-from-dataframe-using-pandas-drop/
elementary = elementary.drop(['First Name', 'Last Name', 'Gender', 'Age', 'Times Available', 'Grade'], axis = 1)
middle = middle.drop(['First Name', 'Last Name', 'Gender', 'Age', 'Times Available', 'Grade'], axis = 1)
display(elementary.head())

,ID,First Hour,Second Hour,Third Hour,Fourth Hour,Trouble,Disbility
2,2,1,1,1,0,7,0
3,3,1,1,1,1,6,1
6,6,1,1,0,1,9,0
7,7,1,1,1,0,8,0
10,10,1,1,1,1,5,1


In [19]:
display(middle.head())

,ID,First Hour,Second Hour,Third Hour,Fourth Hour,Trouble,Disbility
0,0,1,1,1,1,10,1
1,1,1,0,1,1,10,1
4,4,0,1,1,0,7,1
5,5,1,0,0,0,1,1
8,8,0,0,0,1,4,1
